# Generate MACD using daily data service - C#

### Overview
*Moving Average Convergence/Divergence Oscillator* was developed by Gerald Appel and represents the difference between two moving averages of different lengths.

Commonly used MACD the 12-day EMA less the 26-day EMA, but the parameters can be adapted when required. As its name implies, MACD is all about convergence and divergence of the two moving averages

MACD oscilaltes around the zero line as follows:
* positive values increase as the shorter moving average diverges further from the longer one
* negative values indicate that the shorter moving average is below the longer one

This sample demonstrates how to request and plot **on-demand** *Moving Average Convergence/Divergence Oscillator - MACD* from a daily data service.

### Inputs/outputs
Moving Average Convergence/Divergence Oscillator's request require instrument's identifier, date time intervals and number of periods of each moving average as per inputs.

*MACD* is computed as follows:

$$ MACD = 12day,EMA - 26day,EMA $$

This sample shows how to plot a simple graph for basis technical analysis using an open source library. 

### Services used
This sample uses *gRPC requests* in order to retrieve MACD indicator from the hosted service. The queried endpoint in this script are:
* *DailyMacdService*: to directly retrieve daily data from the server

### Packages required
1. Systemathics packages:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.DailyAnalytics.V1*
2. Open source packages
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package

***

# Run daily MACD sample

### Step 1: Install packages

In [ ]:
#i "nuget: file:///home/jovyan/.nuget/packages/"
#i "nuget: https://api.nuget.org/v3/index.json"
#r "nuget: Systemathics.Apis"
#r "nuget: XPlot.Plotly.Interactive"

In [ ]:
using Systemathics.Apis.Helpers;
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.DailyAnalytics.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
using XPlot.Plotly;
using XPlot.Plotly.Interactive;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Get token as metadata
var headers = TokenHelpers.GetTokenAsMetaData();

// Create communication channel
var channel = ChannelHelpers.GetChannel();

### Step 3: Retrieve data

#### 3.1 Request Parameters
To request *daily MACD* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* The MACD parameters

In [ ]:
// Set instrument
var ticker = "AAPL";
var exchange = "XNGS";

#### 3.2 MACD parameters

In [ ]:
// Set EMAs lengths
var longEma = 26;
var shortEma = 12;

#### 3.3 Time period selection

In [ ]:
// Set data interval (we are using Google date time format)
var dateIntervals = new DateInterval()
{
  StartDate = new Date { Year = 2019, Month = 01, Day = 01 },
  EndDate = new Date { Year = 2020, Month = 12, Day = 31 }
};

// Generate constraints based on the previous date selection
// Time constraints are not neeeded for the splits
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);

#### 3.4 MACD request creation
The following code snippet creates *gRPC client*, process daily MACD requests and returns the reply.

In [ ]:
// Instantiate the daily MACD service
var service = new DailyMacdService.DailyMacdServiceClient(channel);

// Create and process the daily MACD request
var request = new DailyMacdRequest 
{ 
    Identifier = new Identifier { Exchange = exchange, Ticker = ticker }, 
    Short = shortEma, 
    Long = longEma,
    Constraints = constraints
};
var reply = service.DailyMacd(request, headers);

// Visualize requests' results
//display(dailyMacdReply.Data)

### Step 4: Visualize data

#### 4.1 Select data

In [ ]:
// Extract the price and the macd
var prices = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Value));
var macdData = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Macd));
var longEma = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Long));
var shortEma = reply.Data.Select(t => new Tuple<DateTime, double>(new DateTime(t.Date.Year, t.Date.Month, t.Date.Day), t.Short));

#### 4.2 Plot MACD and daily prices
Plot MACD request results with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package. 

In [ ]:
var macdGraph = new List<Scattergl>
{
    new Scattergl
    {
        name = "MACD",
        mode = "lines",
        x = macdData.Select(c => c.Item1),
        y = macdData.Select(c => c.Item2),
        yaxis = "y2"
    },
    new Scattergl
    {
        name = "Price",
        mode = "lines",
        x = prices.Select(c => c.Item1),
        y = prices.Select(c => c.Item2)
    },
    new Scattergl
    {
        name = "Long EMA",
        mode = "lines",
        x = longEma.Select(c => c.Item1),
        y = longEma.Select(c => c.Item2)
    },
    new Scattergl
    {
        name = "Short EMA",
        mode = "lines",
        x = shortEma.Select(c => c.Item1),
        y = shortEma.Select(c => c.Item2)
    }
};

var layout = new Layout.Layout{ 
    title = $"MACD and daily prices for {ticker}-{exchange}",
    yaxis = new Yaxis {
        title = "Price", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "MACD", 
        autorange = true,
        side = "right",  
    },
};
var chart = Chart.Plot(macdGraph, layout);
chart.WithXTitle($"Date");
chart.Width = 1500;
chart.Height = 800;
display(chart);